### Replicate the results from Hetland et al. (2021)
We consider the new multivariate Dynamic Conditional Eigenvalue GARCH model (λ-MGARCH) introduced in Hetland et al. (2022 JoE). To ensure the cridability of our Eigenvalue GARCH function, we replicate the results of Table 1, in Hetland et. al.(2021)

In [3]:
# Import Packages
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from scipy.special import gamma
import yfinance as yf
from matplotlib import pyplot as plt 
from numpy.linalg import eig
import Eigen_GARCH as Eigen_GARCH

In [4]:
# Import Data from Yahoo Finance, adjusted closing prices 
# Bank of America corp.(BAC),  JPMorgan Chase & co.(JPM), and Wells Fargo (WFC).
df = yf.download("BAC JPM WFC", start="2006-01-03", end="2018-01-02",group_by="ticker")

# Calculate log-returns for GARCH analysis
data = {"BAC returns":(np.log(df["BAC"]["Adj Close"])-np.log(df["BAC"]["Adj Close"].shift(1)))*100,
      "JPM returns":(np.log(df["JPM"]["Adj Close"])-np.log(df["JPM"]["Adj Close"].shift(1)))*100,
      "WFC returns":(np.log(df["WFC"]["Adj Close"])-np.log(df["WFC"]["Adj Close"].shift(1)))*100}
data = pd.DataFrame.from_dict(data).dropna()

[*********************100%***********************]  3 of 3 completed


In [5]:
# Divid data set into a dataset for initial estimation and a dataset for forecasting
data_est = data.loc[:"2018-01-02"]
data_forecast = data

# Timestamp 
x = np.array(data.reset_index()["Date"])

# Covariance of the dataset
covdata = np.cov(data_est,bias=True)

# Convert dataframe to numpy array
data = np.array(data_est, dtype=np.float64).transpose()

data_forecast = np.array(data_forecast, dtype=np.float64).transpose()


In [6]:
# Set up bounds and lambdas
T = data.shape[1]
k = 3
eigenvalues, eigenvectors = eig(covdata)
lambdas_eig = eigenvalues
bnds = [(-np.pi/2, np.pi/2)]
bnds_eig = [(0, np.pi)]

for i in range(2*(k**2)+k+2):
    bnds_eig.append((0, 100))

# Initial values
arguments_eig = (np.asarray(data),lambdas_eig)
startingVals_eig = [np.random.uniform(0.01,0.2) for i in range(2*(k**2) + k + 3)]

# Estimation
estimation = minimize(Eigen_GARCH.eigenLikelihood , x0=startingVals_eig , 
                          args=arguments_eig , method="SLSQP", bounds=bnds_eig, 
                          options={"maxiter":100000, "ftol":10e-14})

/Users/silkesommerfugl/Desktop/Master/Eigen_GARCH.py:39: RuntimeWarning: overflow encountered in matmul
  lambdas[:,t:t+1] = W + A @ np.multiply(Xtilde[:,t-1:t],Xtilde[:,t-1:t]) + B @ lambdas[:,t-1:t]
/Users/silkesommerfugl/Desktop/Master/Eigen_GARCH.py:42: RuntimeWarning: divide by zero encountered in log
  lls1 = np.log(lambdas[0,:]) + np.multiply(Xtilde[0,:],Xtilde[0,:]) / lambdas[0,:]
/Users/silkesommerfugl/Desktop/Master/Eigen_GARCH.py:42: RuntimeWarning: divide by zero encountered in true_divide
  lls1 = np.log(lambdas[0,:]) + np.multiply(Xtilde[0,:],Xtilde[0,:]) / lambdas[0,:]
/Users/silkesommerfugl/Desktop/Master/Eigen_GARCH.py:42: RuntimeWarning: invalid value encountered in add
  lls1 = np.log(lambdas[0,:]) + np.multiply(Xtilde[0,:],Xtilde[0,:]) / lambdas[0,:]
/Users/silkesommerfugl/Desktop/Master/Eigen_GARCH.py:43: RuntimeWarning: divide by zero encountered in log
  lls2 = np.log(lambdas[1,:]) + np.multiply(Xtilde[1,:],Xtilde[1,:]) / lambdas[1,:]
/Users/silkesommerfugl/Deskt

In [7]:
# Save the estimated parameter values
phi1_eig, phi2_eig, phi3_eig, w1_eig, w2_eig, w3_eig , a11_eig , a12_eig , a13_eig, a21_eig , a22_eig, a23_eig, a31_eig , a32_eig, a33_eig , b11_eig , b12_eig, b13_eig , b21_eig , b22_eig, b23_eig, b31_eig , b32_eig, b33_eig = estimation.x

# Find the llh and information criteria for the Eigen GARCH
llh = - estimation.fun
AIC = 2 * 24 - 2 * (llh)
BIC = - 2 * llh + np.log(T) * 24

# Load into matrices
W_eig = np.array(([w1_eig, w2_eig, w3_eig]))
A_eig = np.array(([a11_eig, a12_eig, a13_eig],[a21_eig, a22_eig, a23_eig],[a31_eig, a32_eig, a33_eig]))
B_eig = np.array(([b11_eig, b12_eig, b13_eig],[b21_eig, b22_eig, b23_eig],[b31_eig, b32_eig, b33_eig]))
V_eig = np.array(([np.cos(phi1_eig), np.sin(phi1_eig),0], [-np.sin(phi1_eig), np.cos(phi1_eig),0 ], [0, 0,1 ]))@ np.array(([np.cos(phi2_eig), 0, np.sin(phi2_eig)], [0, 1,0 ], [-np.sin(phi2_eig), 0, np.cos(phi2_eig)]))@ np.array(( [1, 0,0 ], [0,np.cos(phi3_eig), np.sin(phi3_eig)], [0,-np.sin(phi3_eig), np.cos(phi3_eig)]))

print(f'W: {W_eig}')
print(f'A: {A_eig}')
print(f'B: {B_eig}')
print(f'V: {V_eig}')
print(f'Log-likelihood: {llh}')
print(f'AIC: {AIC}')
print(f'BIC: {BIC}')

W: [0.09178649 0.0305803  0.03793304]
A: [[0.13293651 0.14050609 0.01028771]
 [0.05172401 0.13105591 0.004417  ]
 [0.06703434 0.01585665 0.06447646]]
B: [[7.48327459e-13 2.00984424e-01 3.95755372e-02]
 [4.81718234e-12 6.46042722e-01 3.48056425e-03]
 [8.17149260e-13 1.04997813e-12 9.27414059e-01]]
V: [[ 0.70584615 -0.25668826  0.66022144]
 [-0.22001693  0.8064948   0.54877928]
 [-0.67333035 -0.53261363  0.51278548]]
Log-likelihood: -14889.880249102902
AIC: 29827.760498205804
BIC: 29972.064840518768


In [8]:
phi1_eig, phi2_eig, phi3_eig

(0.3021619272827094, 0.7387040871448453, 0.8043629511576684)